In [1]:
import os
import pandas as pd
from pyensembl import EnsemblRelease

In [2]:
snakefile_path = os.getcwd() + "/../Snakefile"
snakefile_path

'/data/nasif12/home_if12/mumichae/Documents/workspace/ALS/workflow/notebooks/../Snakefile'

In [3]:
try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args

    snakemake = load_rule_args(
        snakefile = snakefile_path,
        rule_name = 'ppi_preprocess_string',
        root="../",
    )

INFO:snakemk_util.rule_args:root dir: /data/nasif12/home_if12/mumichae/Documents/workspace/ALS/workflow/notebooks/../../


In [4]:
release_no = snakemake.params['release']
input_string_db = snakemake.input['db']
output_string_db = snakemake.output['db']

In [5]:
string_db = pd.read_csv(input_string_db, sep = ' ')

In [6]:
string_db.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000272298,0,0,332,62,181,0,125,490
1,9606.ENSP00000000233,9606.ENSP00000253401,0,0,0,0,186,0,56,198
2,9606.ENSP00000000233,9606.ENSP00000401445,0,0,0,0,159,0,0,159
3,9606.ENSP00000000233,9606.ENSP00000418915,0,0,0,61,158,0,542,606
4,9606.ENSP00000000233,9606.ENSP00000327801,0,0,0,88,78,0,89,167


In [7]:
string_db.protein1 = string_db.protein1.str.split('.').str[1]
string_db.protein2 = string_db.protein2.str.split('.').str[1]

In [8]:
ens = EnsemblRelease(release_no)

In [9]:
# Test if retrieving protein ID is possible

ens.gene_id_of_protein_id(string_db.protein1[0])

'ENSG00000004059'

In [10]:
def convert_to_gene(prot):
    try:
        return ens.gene_id_of_protein_id(prot)
    except:
        return None

In [11]:
string_db['gene1'] = string_db.protein1.apply(convert_to_gene)

In [12]:
string_db['gene2'] = string_db.protein2.apply(convert_to_gene)

## Save processed STRING database

In [13]:
string_db.to_csv(output_string_db, sep = '\t', index=False, compression='gzip')

## Explore dataset

In [14]:
number_of_interactions = string_db.shape[0]

In [15]:
string_db.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,gene1,gene2
0,ENSP00000000233,ENSP00000272298,0,0,332,62,181,0,125,490,ENSG00000004059,ENSG00000143933
1,ENSP00000000233,ENSP00000253401,0,0,0,0,186,0,56,198,ENSG00000004059,ENSG00000131089
2,ENSP00000000233,ENSP00000401445,0,0,0,0,159,0,0,159,ENSG00000004059,ENSG00000178607
3,ENSP00000000233,ENSP00000418915,0,0,0,61,158,0,542,606,ENSG00000004059,ENSG00000147889
4,ENSP00000000233,ENSP00000327801,0,0,0,88,78,0,89,167,ENSG00000004059,ENSG00000185624


In [16]:
string_db.dropna().shape[0]

11687788

In [17]:
number_of_interactions

11759454

In [18]:
string_db.query('combined_score > 900')

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,gene1,gene2
15,ENSP00000000233,ENSP00000432568,0,0,0,62,63,900,94,909,ENSG00000004059,ENSG00000173898
16,ENSP00000000233,ENSP00000427900,0,0,0,0,131,900,56,910,ENSG00000004059,ENSG00000197892
40,ENSP00000000233,ENSP00000354878,0,0,0,0,131,900,56,910,ENSG00000004059,ENSG00000139116
52,ENSP00000000233,ENSP00000405926,0,0,0,63,0,900,80,906,ENSG00000004059,ENSG00000134970
78,ENSP00000000233,ENSP00000314615,0,0,0,82,262,900,624,971,ENSG00000004059,ENSG00000101199
...,...,...,...,...,...,...,...,...,...,...,...,...
11759246,ENSP00000485663,ENSP00000268661,0,0,0,192,0,900,49,916,ENSG00000100129,ENSG00000140986
11759247,ENSP00000485663,ENSP00000472985,0,0,0,500,755,900,67,987,ENSG00000100129,ENSG00000083845
11759248,ENSP00000485663,ENSP00000270625,0,0,0,252,731,900,140,980,ENSG00000100129,ENSG00000142534
11759251,ENSP00000485663,ENSP00000403172,0,0,0,499,0,900,0,947,ENSG00000100129,ENSG00000089009
